In [1]:
from glob import glob
import pandas as pd
import json

In [2]:
df = pd.read_parquet('1M-GPT4-Augmented.parquet')

In [3]:
df.head()

,id,system_prompt,question,response
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...
3,t0.408370,You are an AI assistant. You will be given a t...,Please answer the following question: I want t...,"Based on the passage, discuss the primary moti..."
4,cot.86217,You are an AI assistant that helps people find...,James runs a TV show and there are 5 main char...,"James pays the minor characters $15,000 each e..."


In [4]:
files = glob('*.splitted.requested')
translated = {}

for f in files:

    with open(f) as fopen:
        for l in fopen:

            data = json.loads(l)
            if len(data['r']['result']):
                translated[data['src']] = data['r']['result']
            
len(translated)

1742305

In [5]:
df['question'].iloc[0]

"You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:"

In [6]:
from tqdm import tqdm

questions = []
responses = []
for i in tqdm(range(len(df))):
    questions.append(translated.get(df['question'].iloc[i]))
    responses.append(translated.get(df['response'].iloc[i]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 994896/994896 [00:07<00:00, 131926.30it/s]


In [7]:
df['question_ms'] = questions
df['response_ms'] = responses

In [8]:
df.to_parquet('1M-GPT4-Augmented.translated.parquet', index = False)

In [9]:
df

,id,system_prompt,question,response,question_ms,response_ms
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp...","Anda akan diberi definisi tugas dahulu, kemudi...","[\n [""AFC Ajax (amatur)"", ""mempunyai tanah"",..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...,Hasilkan ayat kira-kira lima belas patah perka...,Midsummer House ialah restoran Cina dengan har...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...,Apa yang berlaku seterusnya dalam perenggan in...,C. Dia kemudian mencelupkan jarum dalam dakwat...
3,t0.408370,You are an AI assistant. You will be given a t...,Please answer the following question: I want t...,"Based on the passage, discuss the primary moti...",Sila jawab soalan berikut: Saya ingin menguji ...,"Berdasarkan petikan tersebut, bincangkan motiv..."
4,cot.86217,You are an AI assistant that helps people find...,James runs a TV show and there are 5 main char...,"James pays the minor characters $15,000 each e...",James menjalankan rancangan TV dan terdapat 5 ...,"James membayar watak-watak kecil $15,000 setia..."
...,...,...,...,...,...,...
994891,t0.68400,You are an AI assistant. You will be given a t...,Pick the option in line with common sense to a...,C. person's head\n\nA human eye is a vital sen...,Pilih pilihan selaras dengan akal fikiran untu...,C. kepala orang\n\nMata manusia ialah organ de...
994892,t0.1556271,You are an AI assistant that follows instructi...,Please answer the following question: Read the...,B: could do it much good,Sila jawab soalan berikut: Baca artikel dan pi...,B: boleh melakukannya dengan baik
994893,t0.1213189,You are an AI assistant. You will be given a t...,Suggest a movie title for the following movie ...,"""Rise of the Hooded Rebellion: The Earl's Crus...",Cadangkan tajuk filem untuk plot filem berikut...,"""Kebangkitan Pemberontakan Berkerudung: Perang..."
994894,niv.255896,You are an AI assistant. Provide a detailed an...,Instructions: In this task you are given a lis...,To find the absolute value of the difference b...,Arahan: Dalam tugasan ini anda diberi senarai ...,Untuk mencari nilai mutlak perbezaan antara se...
